<a href="https://colab.research.google.com/github/kmuenpra/EECE7268/blob/main/EECE_CS_7268_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EECE / CS 7268: Verifiable Machine Learning -- Fall 2025
# HW#3: Reachability Analysis of Neural Feedback Loops

**Submission Instructions:** Please upload a .ipynb file (including your code, plots, and written answers) to Gradescope. This file can be updated unlimited times until the submission deadline.

The goal of this assignment is to give you hands-on experience with reachability analysis of neural feedback loops. You will code some forward and backward reachability algorithms, leveraging existing NN verification libraries. Depending on your interests, you are encouraged to try implementing some sort of extension for the extra credit problem.

You're welcome to use Jax, PyTorch, TensorFlow, Keras, or other appropriate libraries to help make your life easier throughout the problems - we provide examples based on `jax_verify` but tried to make the function signatures somewhat framework-agnostic.

Please enter your contributions here! Mention if you have collaborated with someone on this homework or if you have used any AI tools for help with this homework! If this is not filled out, the assignment will be given a zero.

# Dependencies, Imports, & Useful Functions

In [ ]:
!apt install libgmp-dev libcdd-dev
!pip install torch \
  torchvision \
  packaging>=20.0 \
  pytest==8.1.1 \
  pylint>=2.15 \
  pytest-order>=1.0.0 \
  pytest-mock>=3.14 \
  appdirs>=1.4 \
  pyyaml>=5.0 \
  tqdm>=4.64 \
  graphviz>=0.20.3 \
  matplotlib \
  pandas \
  tabulate \
  colour \
  jax==0.5.3 \
  parameterized \
  pypoman \
  alphashape \
  scikit-learn \
  scipy \
  imageio \
  tqdm \
  pyclipper \
  pygifsicle \
  mypy \
  pylint
!pip install --no-deps "auto-lirpa @ git+https://gitlab.com/neu-autonomy/certifiable-learning/auto-lirpa@numpy2.0-update-hacks"
!pip install \
    "jax_verify @ git+https://gitlab.com/neu-autonomy/certifiable-learning/jax_verify@mainline_repo"
!pip install --no-deps "crown_ibp @ git+https://gitlab.com/neu-autonomy/certifiable-learning/crown_ibp.git"
!pip install --no-deps "nfl_veripy @ git+https://github.com/neu-autonomy/nfl_veripy.git"

In [ ]:
import argparse  # noqa: E402
import ast  # noqa: E402
import logging  # noqa: E402
import os  # noqa: E402
import time  # noqa: E402
from typing import Dict, Tuple  # noqa: E402

import numpy as np  # noqa: E402
import yaml  # noqa: E402

import nfl_veripy.analyzers as analyzers  # noqa: E402
import nfl_veripy.constraints as constraints  # noqa: E402
import nfl_veripy.dynamics as dynamics  # noqa: E402
from nfl_veripy.utils.nn import load_controller  # noqa: E402
from nfl_veripy.utils.utils import get_plot_filename  # noqa: E402

import torch
import numpy as np
import cvxpy as cp
from enum import Enum
import matplotlib.pyplot as plt
import jax_verify
import functools
import jax.numpy as jnp
from typing import Callable, Optional
from matplotlib.patches import Rectangle
from itertools import product

Some possibly useful methods for defining a NN in jax, copied in from the last HW assignment:

In [ ]:
def pytorch_model_to_jax(torch_model: torch.nn.Sequential):
  params = []
  act = None

  # Extract params (weights, biases) from torch layers, to be used in
  # jax.
  # Note: This propagator assumes a feed-forward relu NN.
  for m in torch_model.modules():
    if isinstance(m, torch.nn.Sequential):
      continue
    elif isinstance(m, torch.nn.ReLU):
      if act is None or act == "relu":
        act = "relu"
    elif isinstance(m, torch.nn.Linear):
      w = m.weight.data.numpy().T
      b = m.bias.data.numpy()
      params.append((w, b))
  return functools.partial(relu_nn, params)

def relu_nn(params, inputs):
  for W, b in params[:-1]:
    outputs = jnp.dot(inputs, W) + b
    inputs = jnp.maximum(outputs, 0)
  W, b = params[-1]
  return jnp.dot(inputs, W) + b

def jax_interval_to_np_range(interval: jax_verify.IntervalBound) -> np.ndarray:
  return np.vstack([interval.lower, interval.upper]).T

def np_range_to_jax_interval(input_range: np.ndarray) -> jax_verify.IntervalBound:
  return jax_verify.IntervalBound(input_range[:, 0], input_range[:, 1])

Here are some methods as examples of working with the controller and dynamics:

In [ ]:
# Load a dynamics model that implements x_{t+1} = Ax_{t} + Bu_{t} for a ground robot
dyn = dynamics.get_dynamics_instance("GroundRobot", "FullState")
# Load a controller for the ground robot that is trained to avoid an obstacle near the origin
controller = load_controller("GroundRobotSI", model_name="complex_potential_field")
jax_controller = pytorch_model_to_jax(controller)

# Problem 1: Forward Reachability Analysis

Here is an example of how to implement the closed-loop dynamics for 1 step (`step_1`) and plot some randomly sampled ($x_t, x_{t+1})$ pairs (`plot_1step_samples_and_bounds`):

In [ ]:
def step_1(xt):
  ut = jax_controller(xt)
  xt1 = dyn.dynamics_step_jnp(xt, ut)
  return xt1

def plot_1step_samples_and_bounds(initial_state_range: np.ndarray, step_fn: Callable, bounds: Optional[np.ndarray]) -> None:

  xt = np.random.uniform(low=initial_state_range[:, 0], high=initial_state_range[:, 1], size=(1000,2))
  xt1 = step_fn(xt)
  plt.plot(xt[:, 0], xt[:, 1], 'o')
  plt.plot(xt1[:, 0], xt1[:, 1], 'o')
  dims = [0, 1]
  if bounds is not None:
    rect = Rectangle(
        bounds[dims, 0],
        bounds[dims[0], 1] - bounds[dims[0], 0],
        bounds[dims[1], 1] - bounds[dims[1], 0],
        fc="None",
        linewidth=2,
        edgecolor="red",
    )
    plt.gca().add_patch(rect)
  plt.show()

initial_state_range = np.array([[-5.5,-4.5],[.5,1.5]])
plot_1step_samples_and_bounds(initial_state_range, step_1, bounds=None)

## 1a) Calculate outer bounds on the 1-step reachable set

You'll start by implementing a function that calculates outer bounds on the 1-step reachable set. It's up to you how you do this, but as an example, you could do this in 3 lines using `jax_verify`.

**Deliverables**:
- Implement `compute_1step_reachable_set`
- Plot your computed reachable set bounds over the Monte Carlo samples

In [ ]:
def compute_1step_reachable_set(initial_state_range: np.ndarray, step_fn: Callable) -> np.ndarray:

  # - initial_state_range: np.array of size (num_states, 2) containing min/max per state
  # - step_fn: function that takes 1 argument, x_t, and outputs the subsequent state, x_{t+1}
  #
  # Returns:
  # - next_state_range: np.array of same shape as initial_state_range, but corresponding to the next timestep

  # TODO: implement this

  return next_state_range

output_range = compute_1step_reachable_set(initial_state_range, step_1)
print(f"1-step Reachable Set: {output_range}.")

# You can pass the computed output_range to the function we defined before
# and it will draw your bound as a rectangle over the sampled points:
plot_1step_samples_and_bounds(initial_state_range, step_1, bounds=output_range)

## 1b) Calculate outer bounds on the reachable sets for T steps into the future: Iterative Version

Now that you have a 1-step reachable set algorithm, you will extend this to compute reachable sets T steps into the future.

**Deliverables**:
- Implement `compute_Tstep_reachable_set_iterative`, an iterative algorithm to get reachable set outer bounds for timesteps $(1, 2, ..., T)$.



To help out, we provide you with a function to plot those samples (and bounds, for later parts of the problem), so you can see what the forward reachable sets should roughly look like:

In [ ]:
def plot_Tstep_samples_and_bounds(initial_state_range: np.ndarray, step_1: Callable, num_steps: int, bounds: Optional[np.ndarray]) -> None:

  # Sample a bunch of points in the initial state set, then run those forward T steps and plot
  xt = np.random.uniform(low=initial_state_range[:, 0], high=initial_state_range[:, 1], size=(1000,2))
  plt.plot(xt[:, 0], xt[:, 1], '.')
  for _ in range(num_steps):
    xt = step_1(xt)
    plt.plot(xt[:, 0], xt[:, 1], '.')

  # If passed in, draw the bounds as a rectangle at each timestep
  if bounds is not None:
    for bound in bounds:
      dims = [0, 1]
      rect = Rectangle(
          bound[dims, 0],
          bound[dims[0], 1] - bound[dims[0], 0],
          bound[dims[1], 1] - bound[dims[1], 0],
          fc="None",
          linewidth=2,
          edgecolor="red",
      )
      plt.gca().add_patch(rect)
  plt.show()

initial_state_range = np.array([[-5.5,-4.5],[.5,1.5]])
num_steps = 10
plot_Tstep_samples_and_bounds(initial_state_range, step_1, num_steps, bounds=None)

Now that you have visualized what the reachable sets should roughly look like, you will implement an algorithm to compute outer bounds on those reachable sets.

Your first implementation will compute reachable sets iteratively, as in:

- $\bar{R}_1(X_t) = \texttt{compute_1step_reachable_set}(X_t)$
- $\bar{R}_2(X_t) = \texttt{compute_1step_reachable_set}(\bar{R}_1(X_t))$
- $\ldots$

In [ ]:
def compute_Tstep_reachable_set_iterative(initial_state_range: np.ndarray, step_fn: Callable, num_steps: int) -> np.ndarray:
  #
  # initial_state_range: (num_states, 2) array containing min/max per state
  # step_fn: function that accepts array of state samples at initial timestep and returns list of state samples at times [t+1, ..., t+num_steps]
  # num_steps: non-negative integer
  #
  # Return:
  # - list of state_ranges at times [t+1, t+2, ..., t+num_steps]
  #  i.e., [(num_states, 2)_t=t+1, ..., (num_states, 2)_t=t+T]

  # TODO: implement this using

  return output_range

initial_state_range = np.array([[-5.5,-4.5],[.5,1.5]])
output_ranges = compute_Tstep_reachable_set_iterative(initial_state_range, step_1, num_steps)
plot_Tstep_samples_and_bounds(initial_state_range, step_1, num_steps, bounds=output_ranges)

print('output ranges:')
for output_range in output_ranges:
  print(output_range)

## 1c) Calculate outer bounds on the reachable sets for T steps into the future: One-Shot Version

Iterative reachability analysis can provide excessively conservative bounds, for several reasons. To address this, you will implement one way to get tighter bounds, by instead computing the bounds for all timesteps in one-shot, i.e.,

- $\bar{R}_T(X_t), \ldots, \bar{R}_2(X_t), \bar{R}_1(X_t) = \texttt{compute_Tstep_reachable_set}(X_t, T)$

Remember, if the function you ask `jax_verify` to compute bounds on returns a list of outputs, you will get a list of bounds that corresponds to each output in the list. We've helped you out by implementing `step_T`, which outputs a list of future states for a given initial state. Since `jax_verify` expects the function to verify just has 1 argument (e.g., the initial state), it's helpful to use `functools.partial` to set any other arguments to a particular value (e.g., `num_steps`).

**Deliverables**:
- Implement `compute_Tstep_reachable_set_one_shot`
- Briefly comment on the difference between the two algorithms' bounds and runtimes

In [ ]:
def step_T(num_steps: int, xt: np.ndarray) -> list[np.ndarray]:
  #
  # num_steps: non-negative integer
  # xt: (num_samples, num_states) np array of state samples at time t
  #
  # Return:
  # - list of state samples at times [t+1, t+2, ..., t+num_steps]
  #  i.e., [(num_samples, num_states)_t=t+1, ..., (num_samples, num_states)_t=t+num_steps]
  #
  # jax_verify will by default compute bounds on any "outputs" of the function
  # to verify, so if you return a list of states at each timestep, you'll
  # get bounds on all reachable sets in one call.

  xts = [xt]
  for _ in range(num_steps):
    ut = jax_controller(xts[-1])
    xts.append(dyn.dynamics_step_jnp(xts[-1], ut))
  return xts[1:]

num_steps = 10
step_fn = functools.partial(step_T, num_steps)

step_fn(np.array([[-5., 0.]]))

In [ ]:
def compute_Tstep_reachable_set_one_shot(initial_state_range: np.ndarray, step_fn: Callable, num_steps: int) -> np.ndarray:
  #
  # initial_state_range: (num_states, 2) array containing min/max per state
  # step_fn: function that accepts array of state samples at initial timestep and returns list of state samples at times [t+1, ..., t+num_steps]
  # num_steps: non-negative integer
  #
  # Return:
  # - list of state_ranges at times [t+1, t+2, ..., t+num_steps]
  #  i.e., [(num_states, 2)_t=t+1, ..., (num_states, 2)_t=t+T]

  # TODO: implement this using

  return output_range

output_ranges = compute_Tstep_reachable_set_one_shot(initial_state_range, step_fn, num_steps)
plot_Tstep_samples_and_bounds(initial_state_range, step_1, num_steps, bounds=output_ranges)

print('output ranges:')
for output_range in output_ranges:
  print(output_range)

How do your iterative and one-shot reachable sets compare? The one-shot sets should be tighter, which may become more noticeable after a few timesteps. How did the computation time compare?

## 1d) Try your one-shot forward reachability algorithm on bifurcating dynamics

So far, we considered an initial state set that had positive y component, so the robot went "up" around the obstacle, regardless of where it started within that set. Now, let's see how things look if we move the `initial_state_set` to span y=0.

**Deliverables**:
- Plot of the samples over 10 timesteps (should just be able to run the 1st cell below)
- Plot of the reachable set estimates over 2 timesteps (should just be able to run the 2nd cell below)


Visualize 10 steps of the dynamics via sampling:

In [ ]:
initial_state_range = np.array([[-5.5,-4.5],[-.5,.5]])
plot_Tstep_samples_and_bounds(initial_state_range, step_1, num_steps, bounds=None)

Next, run your one-shot forward reachability algorithm -- just for 1 or 2 timesteps -- to see how well it bounds the "true" reachable set (as estimated by the samples):

In [ ]:
num_steps = 2
step_fn = functools.partial(step_T, num_steps)
output_ranges = compute_Tstep_reachable_set_one_shot(initial_state_range, step_fn, num_steps)
plot_Tstep_samples_and_bounds(initial_state_range, step_1, num_steps, bounds=output_ranges)

The bounds will probably be quite loose. We will use backward reachability in the next problem to try to get a more useful answer.

# Problem 2: Backward Reachability Analysis

Sometimes, the forward reachable set over-approximations are way too loose to be useful, so we need a new strategy. Or other times, it may not even be important to explicitly compute the forward reachable sets. For example, if we simply want to verify that the system will not enter some part of the state space (e.g., for obstacle avoidance), we could try to directly verify that property instead.

In this problem, instead of computing forward reachable sets starting from the `initial_state_set`, we will solve the backward reachability problem. First, we will define a `target_state_set`, which describes the states we do not want our system to ever enter.

Then, we will define the `backprojection_sets` as all the states that would lead to the `target_state_set`, when following the system's dynamics and trained control policy. Since it will be intractable to compute the `backprojection_sets` exactly, we will aim to compute outer bounds on these sets.

How is this useful? Once we have these `backprojection_sets` computed, right before we let our robot start driving, we can check that it is not starting in one of the `backprojection_set` over-approximations, and if it isn't, we have a proof that the system will not enter the `target_state_set`. For an even stronger result, if you can prove that the first backprojection set is inside the target set, it follows that as long as the system starts outside of the target set, the system will never enter the target set.

## 2a) Compute "true" backprojection sets by sampling

We are trying to compute the backprojection set but it's difficult to know what this "should" look like. To get an idea (and an under-approximation), sample a bunch of points from the state space and check if the subsequent state (i.e., after running that state through the closed-loop dynamics for 1) will be inside the target set. As a hint, all of these points should be inside the target set, since this is a well-trained collision avoidance policy.

**Deliverables**:
- Implement `estimate_true_backprojection_set`
- Plot the "true" (sampling-based) backprojection set and the target set

In [ ]:
target_state_set = np.array([[-1.0, 1.0], [-1.0, 1.0]])*0.2

def get_true_backprojection_set(target_state_set: np.ndarray, step_fn: Callable) -> np.ndarray:

  # Return:
  # - np.array of samples within $P_1(X_t)$ (e.g., if only 13 of your 10000 samples end up in the target set, you'd return an array of size (13,2))

  # TODO: Implement this

  return true_backprojection_set

true_backprojection_set = get_true_backprojection_set(target_state_set, step_1)

plt.plot(true_backprojection_set[:, 0], true_backprojection_set[:, 1], '.')
rect = Rectangle(
    target_state_set[[0, 1], 0],
    target_state_set[0, 1] - target_state_set[0, 0],
    target_state_set[1, 1] - target_state_set[1, 0],
    fc="None",
    linewidth=2,
    edgecolor="red",
)
plt.gca().add_patch(rect)
plt.show()

## 2b) Compute backreachable set using LPs

Recall that for some target set, $X_t$, the backreachable set, $R_1(X_t)$, is the set of states for which *some* control within the control limits would lead the system into the target set:

$R_1(X_t) = \{ x_{t-1} | Ax_{t-1} + Bu_{t-1} \in X_t, u_{t-1} \in U \}$

Here, you'll compute rectangular outer bounds on $R_1(X_t)$ by setting up some LPs, and you'll then use these bounds in subsequent problems.

**Deliverables**:
- Implement `get_backreachable_set`
- Plot the rectangular bounds on the backreachable set, along with the target set and the "true" backprojection set from before

In [ ]:
def get_backreachable_set(target_state_set: np.ndarray, dyn: dynamics.Dynamics) -> np.ndarray:

  # Hint: you may want to use dyn.At, dyn.bt, dyn.u_limits, or dyn.dynamics_step
  # Hint: you may want to use cvxpy for this

  # TODO: Implement this

  return backreachable_set

backreachable_set = get_backreachable_set(target_state_set, dyn)

plt.plot(true_backprojection_set[:, 0], true_backprojection_set[:, 1], '.')
rect = Rectangle(
    target_state_set[[0, 1], 0],
    target_state_set[0, 1] - target_state_set[0, 0],
    target_state_set[1, 1] - target_state_set[1, 0],
    fc="None",
    linewidth=2,
    edgecolor="red",
)
plt.gca().add_patch(rect)
rect = Rectangle(
    backreachable_set[[0, 1], 0],
    backreachable_set[0, 1] - backreachable_set[0, 0],
    backreachable_set[1, 1] - backreachable_set[1, 0],
    fc="None",
    linewidth=2,
    edgecolor="black",
)
plt.gca().add_patch(rect)
plt.show()

## 2c) Compute backprojection set using LPs

In class we derived 2 ways to refine the `backreachable_set` using CROWN. You are welcome to implement either version.

- Version 1: Relax `step_1` over $\bar{R}_1(X_t)$ to get $M, n$ such that $Mx_{t-1}+n \leq Cx_t$, to get the polytope description of $\bar{P}_1(X_t)$ without solving an LP (for this HW, you probably should inflate this to a rectangle to make plotting & partitioning easier)
- Version 2: Relax `jax_controller` over $\bar{R}_1(X_t)$ to get to get $\bar{M}, \bar{n}, \bar{M}', \bar{n}'$ such that $\bar{M}'x_{t-1}+\bar{n}' \leq \pi(x_t) \leq \bar{M}x_{t-1}+\bar{n}$, then add these inequalities as additional constraints to the LP you used to get $\bar{R}_1(X_t)$.

Hint: You may want to use `backward_crown_bound_propagation_linfun`, which is a small modification to `jax_verify` to output the slope/offset of the linear bounds rather than the min/max over the domain.

**Deliverables**:
- Implement `get_backprojection_set`
- Plot the BP, BR, and "true" BP sets together

In [ ]:
from nfl_veripy.utils.closed_loop_verification_jax import backward_crown_bound_propagation_linfun

# Hint: to get the linear CROWN bounds from jax_verify:
input_bounds = np_range_to_jax_interval(backreachable_set)
obj = jnp.vstack((jnp.eye(dyn.num_inputs), -jnp.eye(dyn.num_inputs)))
linfuns = backward_crown_bound_propagation_linfun(jax_controller, input_bounds, obj=obj)

# Then linfuns[0].lin_coeffs and linfuns[0].offset will contain the slope/offset from state to control
# Hint: You'll need to think about the signs of the entries in obj & linfuns to construct the inequalities properly

In [ ]:
def get_backprojection_set(target_state_set: np.ndarray, dyn: dynamics.Dynamics, backreachable_set: np.ndarray, controller: Callable) -> np.ndarray:

  # Hint: you may want to use dyn.At, dyn.bt, dyn.u_limits, or dyn.dynamics_step
  # Hint: you may want to use cvxpy for this

  # TODO: Implement this

  return backreachable_set

backprojection_set = get_backprojection_set(target_state_set, dyn, backreachable_set, jax_controller)

plt.plot(true_backprojection_set[:, 0], true_backprojection_set[:, 1], '.')
rect = Rectangle(
    target_state_set[[0, 1], 0],
    target_state_set[0, 1] - target_state_set[0, 0],
    target_state_set[1, 1] - target_state_set[1, 0],
    fc="None",
    linewidth=2,
    edgecolor="red",
)
plt.gca().add_patch(rect)
rect = Rectangle(
    backreachable_set[[0, 1], 0],
    backreachable_set[0, 1] - backreachable_set[0, 0],
    backreachable_set[1, 1] - backreachable_set[1, 0],
    fc="None",
    linewidth=2,
    edgecolor="black",
)
plt.gca().add_patch(rect)
rect = Rectangle(
    backprojection_set[[0, 1], 0],
    backprojection_set[0, 1] - backprojection_set[0, 0],
    backprojection_set[1, 1] - backprojection_set[1, 0],
    fc="None",
    linewidth=2,
    edgecolor="green",
)
plt.gca().add_patch(rect)
plt.show()

It is expected that your `backprojection_set` is the same as your `backreachable_set` at this point. This happens when the constrains induced by the CROWN relaxation are looser than the bounds induced by the control limits. You'll address this in the next part.

## 2d) Branch and Bound: Uniform Splitting

One strategy to (hopefully) get a tighter CROWN relaxation is to individually consider subsets regions of the backreachable set, then put them back together.

We provide a uniform partitioning strategy that splits the `backreachable_set` into subsets and runs your `get_backprojection_set` on each subset iteratively. The union of these BP sets should provide a much better outer bound on $P_1(X_t)$ than you had before!

Note: it's ok if some of the BP subsets are empty!

**Deliverables**:
- Run the cell below to see if you get meaningful bounds on $P_1(X_t)$
- Experiment with different numbers of partitions and show your results

In [ ]:
num_partitions = [2, 2]

slope = np.divide(
    (backreachable_set[:, 1] - backreachable_set[:, 0]), num_partitions
)
for element in product(
    *[range(num) for num in num_partitions]
):
    element_subset = np.array(element)
    backreachable_set_subset = np.empty_like(backreachable_set)
    backreachable_set_subset[..., 0] = backreachable_set[..., 0] + np.multiply(
        element_subset, slope
    )
    backreachable_set_subset[..., 1] = backreachable_set[..., 0] + np.multiply(
        element_subset + 1, slope
    )
    print(backreachable_set_subset)

    backprojection_set_subset = get_backprojection_set(target_state_set, dyn, backreachable_set_subset, jax_controller)

    rect = Rectangle(
        backprojection_set_subset[[0, 1], 0],
        backprojection_set_subset[0, 1] - backprojection_set_subset[0, 0],
        backprojection_set_subset[1, 1] - backprojection_set_subset[1, 0],
        fc="None",
        linewidth=2,
        edgecolor="green",
    )
    plt.gca().add_patch(rect)

plt.plot(true_backprojection_set[:, 0], true_backprojection_set[:, 1], '.')
rect = Rectangle(
    target_state_set[[0, 1], 0],
    target_state_set[0, 1] - target_state_set[0, 0],
    target_state_set[1, 1] - target_state_set[1, 0],
    fc="None",
    linewidth=2,
    edgecolor="red",
)
plt.gca().add_patch(rect)
rect = Rectangle(
    backreachable_set[[0, 1], 0],
    backreachable_set[0, 1] - backreachable_set[0, 0],
    backreachable_set[1, 1] - backreachable_set[1, 0],
    fc="None",
    linewidth=2,
    edgecolor="black",
)
plt.gca().add_patch(rect)
plt.show()

# Problem 3: Extend/Improve the above algorithms in some way (Extra Credit)

There are lots of ways to potentially improve the baseline implementations  above. You can get extra credit by trying something interesting to improve the results (very open-ended). Some possible ideas:

- Implement whichever version of `get_backprojection_set` you didn't implement before to see how the results differ
- Use branch & bound to tighten the forward reachable sets on the bifurcating example (how should the branches be performed?)
- Train a new control policy that enables tighter analysis ([here](https://github.com/neu-autonomy/nfl_veripy/blob/7d2011d8800e5951d6ddc67d0674c66553523d58/src/nfl_veripy/utils/controller_generation.py#L98) is the function used to generate the controller used in this notebook)
- If using `jax_verify`, get JIT compilation to work and see how fast the results are ([here](https://github.com/google-deepmind/jax_verify/blob/fe8ea3c8da81350ca92203a5089d37c2b93f8583/jax_verify/tests/bound_propagation_test.py#L171) is an example of how to do this)
- Somehow apply these methods to the context of RL (e.g., define a reward function for the task and bound the set of possible accumulated rewards)
- Incorporate some noisy observation process (e.g., add uniform noise at each timestep within some known bounds) into the step function and bound calculation

All done!